In [ ]:
import pandas as pd

df = pd.read_parquet("01_preprocess/building_real__all_devices__2023_dirty.parquet")
# df.index = pd.DatetimeIndex(pd.to_datetime(df["Time"]))
df.head()

In [ ]:
df["solar_generation"].mask(df["solar_generation"] < 0).fillna(0.0).plot.line()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


df_viz = df.copy()
df_viz["solar_generation"] = df_viz["solar_generation"].mask(df["solar_generation"] < 0).fillna(0.0).fillna(0.0) / 1000
df_viz["non_shiftable_load"] = df_viz["non_shiftable_load"] / 1000
df_viz = df_viz.rename(columns={"non_shiftable_load": "Building Power Consumption (Real)", "solar_generation": "PV Power (Real)"})
df_long = (
    df_viz.reset_index()
    .melt(
        id_vars="datetime",
        value_vars=["Building Power Consumption (Real)", "PV Power (Real)"],
        var_name="sensor",
        value_name="reading"
    )
)
sns.set_context("paper", font_scale=1.2, rc={"lines.linewidth": 0.8})
g = sns.relplot(
    data=df_long,
    x="datetime", y="reading",
    kind="line",
    col="sensor",
    col_wrap=3,
    facet_kws=dict(sharey=False),
    height=5,
    aspect=1.5
)

for ax in g.axes.flatten():
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
    ax.set_xlabel('')  # drop per‐panel x‐label if desired


g.set_axis_labels("Time", "Power (kWh)")
g.set_titles("{col_name}")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(12,5))
df_filtered = df[df["non_shiftable_load"] < 100]
df_filtered["Building Power Consumption"] = df_filtered["non_shiftable_load"]
sns.lineplot(
    data=df_filtered,
    x="datetime",
    y="Building Power Consumption",
    marker=None
)

full_min = df.index.min()
full_max = df.index.max()
ax = plt.gca()
ax.set_xlim(full_min, full_max)
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("")
ax.set_ylabel("Building Power (kWh)")
ax.set_title("Building Power Consumption (Real)")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df_citylearn = pd.read_csv("datasets/baseline/Building_1.csv")
plt.figure(figsize=(12,5))
df_filtered = df_citylearn.reset_index().rename(columns={"index":"row"})
df_filtered["Building Power Consumption (CityLearn)"] = df_filtered["non_shiftable_load"]
sns.lineplot(
    data=df_filtered,
    x="row",
    # x=None,
    y="Building Power Consumption (CityLearn)",
    marker=None
)

# full_min = df.index.min()
# full_max = df.index.max()
ax = plt.gca()
# # ax.set_xlim(full_min, full_max)
# ax.xaxis.set_major_locator(mdates.AutoDateLocator())
# ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("Timesteps (Hourly)")
ax.set_ylabel("Building Power (kWh)")
ax.set_title("Building Power Consumption (CityLearn)")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df_citylearn = pd.read_csv("datasets/baseline/Building_1.csv")
plt.figure(figsize=(12,5))
df_filtered = df_citylearn.reset_index().rename(columns={"index":"row"})
df_filtered["Building Solar Generation (CityLearn)"] = df_filtered["solar_generation"] / 1000
sns.lineplot(
    data=df_filtered,
    x="row",
    # x=None,
    y="Building Solar Generation (CityLearn)",
    marker=None
)

# full_min = df.index.min()
# full_max = df.index.max()
ax = plt.gca()
# # ax.set_xlim(full_min, full_max)
# ax.xaxis.set_major_locator(mdates.AutoDateLocator())
# ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("Timesteps (Hourly)")
ax.set_ylabel("Solar Generation (kWh)")
ax.set_title("Building Solar Generation (CityLearn)")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(12,5))
df_filtered = df[df["non_shiftable_load"] < 100]
df_filtered["Building Power Consumption"] = df_filtered["non_shiftable_load"][3000:3072]
sns.lineplot(
    data=df_filtered,
    x="datetime",
    y="Building Power Consumption",
    marker=None
)

# full_min = df.index.min()
# full_max = df.index.max()
ax = plt.gca()
# ax.set_xlim(full_min, full_max)
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("")
ax.set_ylabel("Building Power (kWh)")
ax.set_title("Building Power Consumption - 3 Days (Real)")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df_grid = pd.read_csv("00_raw/building_real__2023.csv")
df_grid[df_grid["SMA-3008628305-EDMM_pAcGrid"] > -100000]["SMA-3008628305-EDMM_pAcGrid"].plot.line()

In [ ]:

plt.figure(figsize=(12,5))

mask = df_grid["SMA-3008628305-EDMM_pAcGrid"] > -10000
df_filt = df_grid[mask].copy()

df_filt["Time"] = pd.to_datetime(df_filt["Time"], utc=False).dt.tz_localize("Europe/Dublin", ambiguous=False)
df_filt = df_filt.set_index("Time")
df_filt["Grid Power"] = df_filt["SMA-3008628305-EDMM_pAcGrid"]

sns.lineplot(
    data=df_filt,
    x=df_filt.index,
    y="Grid Power",
    marker=None
)

full_min = df_filt.index.min()
full_max = df_filt.index.max()
ax = plt.gca()
ax.set_xlim(full_min, full_max)
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("")
ax.set_ylabel("Grid Power (kWh)")
ax.set_title("Grid Power (Real)")
plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(12,5))

mask = df_grid["SMA-3008628305-EDMM_pAcGrid"] > -10000
df_filt = df_grid[mask][3000:3072].copy()

df_filt["Time"] = pd.to_datetime(df_filt["Time"], utc=False).dt.tz_localize("Europe/Dublin", ambiguous=False)
df_filt = df_filt.set_index("Time")
df_filt["Grid Power"] = df_filt["SMA-3008628305-EDMM_pAcGrid"]

sns.lineplot(
    data=df_filt,
    x=df_filt.index,
    y="Grid Power",
    marker=None
)

full_min = df_filt.index.min()
full_max = df_filt.index.max()
ax = plt.gca()
ax.set_xlim(full_min, full_max)
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("")
ax.set_ylabel("Grid Power (kWh)")
ax.set_title("Grid Power - 3 Days (Real)")
plt.tight_layout()
plt.show()

In [ ]:
df_filtered = df[df["non_shiftable_load"] < 100]
df_filtered["non_shiftable_load"].plot.line();

In [ ]:
df["10000000846c766e-OM_pAcHome"].isna().sum()

In [ ]:
df["10000000846c766e-OM_pAcHome"].notna().sum()

In [ ]:
df[df["10000000846c766e-OM_pAcHome"] > 10]["10000000846c766e-OM_pAcHome"].plot.line(title="10000000846c766e-OM_pAcHome");

In [ ]:
days = 130
minutes = 24 * 60
time_start = days * minutes
print(time_start)
time_end = time_start + (5 * minutes)
print(time_end)
df[time_start:time_end]["10000000846c766e-OM_pAcHome"].plot.line(title="10000000846c766e-OM_pAcHome");

In [ ]:
df[time_start:time_end]["SMA-3008628305-EDMM_pAcGrid"].plot(title="SMA-3008628305-EDMM_pAcGrid");

In [ ]:
df[time_start:time_end]["SMA-3012104048-SI_sOc"].plot(title="SMA-3012104048-SI_sOc");

In [ ]:
df[time_start:time_end]["SMA-3012104048-SI_pBat"].plot(title="SMA-3012104048-SI_pBat");

In [ ]:
df["10000000846c766e-OM_pAcHome"].plot.line()

In [ ]:
df[time_start:time_end]["SMA-3008628305-EDMM_pAcGrid"].plot.line(title="SMA-3008628305-EDMM_pAcGrid");

In [ ]:
df[time_start:time_end]["SMA-3010507943-STP_pDcPV"].min()

In [ ]:
df[time_start:time_end]["SMA-3010507943-STP_pDcPV"].replace(-2147483648.0, 0).plot.line(title="SMA-3010507943-STP_pDcPV");